In [ ]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import (
    RegionMocked,
    create_fake_regions,
    read_true_geometry,
    read_true_metadata,
)
from electoralyze.region.redistribute import redistribute
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    create_region_mapping_base,
)

In [ ]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("region_b"),
    geometry_to=read_true_geometry("region_c"),
)
intersection_area.pipe(print)

In [ ]:
import shutil
import tempfile

temp_dir = tempfile.TemporaryDirectory(delete=False)
region_ = create_fake_regions(temp_dir.name)
region_
region_.region_a.geometry
region_.region_b.geometry
region_.region_c.geometry
shutil.rmtree(temp_dir.name)

In [ ]:
RegionMocked

In [ ]:
data_by_from = pl.DataFrame({"region_b": ["A"], "data": [1]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.region_b,
    region_to=region_.region_a,
    mapping="intersection_area",
    redistribute_with_full=False,
)

In [ ]:
data_by_from = pl.DataFrame({"region_c": ["X", "Y", "Z"], "data": [1, 1, 1]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.region_c,
    region_through=region_.region_a,
    region_to=region_.region_b,
    mapping="intersection_area",
    redistribute_with_full=False,
)